<a href="https://colab.research.google.com/github/NewTAs/GoogLeNet/blob/main/GoogLeNet_Step4_DriveVer_Epoch50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 참고: https://tykimos.github.io/2017/03/08/CNN_Getting_Started/

In [1]:
# Step1 Code
import os
import re
import glob
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf

from google.colab import drive
drive.mount('googlenet')

# New Cord
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

#np.random.seed(4) # 랜덤시드 고정시키기

Mounted at googlenet


In [2]:
pictureWeightHeightSize = 227

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/googlenet/MyDrive/GoogLeNet_Data/TrainData_Step3/',
        target_size=(pictureWeightHeightSize, pictureWeightHeightSize),
        batch_size=4,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        '/content/googlenet/MyDrive/GoogLeNet_Data/TestData_Step3/',
        target_size=(pictureWeightHeightSize, pictureWeightHeightSize),    
        batch_size=4,
        class_mode='categorical')

Found 60 images belonging to 4 classes.
Found 12 images belonging to 4 classes.


In [3]:
input_layer = tf.keras.layers.Input(shape=(227, 227, 3))

In [4]:
stem_layer = tf.keras.layers.Conv2D(64, (3, 3), strides=(1, 1), padding='SAME', activation='relu')(input_layer)
max_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='SAME')(stem_layer)
stem_layer = tf.keras.layers.Conv2D(192, (3, 3), strides=(1, 1), padding='SAME', activation='relu')(max_pool)
max_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='SAME')(stem_layer)

In [5]:
inception_block3a_1x1 = tf.keras.layers.Conv2D(64, (1, 1), strides=(1, 1), padding='SAME', activation='relu')(max_pool)

inception_block3a_3x3 = tf.keras.layers.Conv2D(96, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block3a_3x3 = tf.keras.layers.Conv2D(128, (3, 3), padding='SAME', activation='relu')(inception_block3a_3x3)

inception_block3a_5x5 = tf.keras.layers.Conv2D(16, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block3a_5x5 = tf.keras.layers.Conv2D(32, (5, 5), padding='SAME', activation='relu')(inception_block3a_5x5)

inception_block3a_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(max_pool)
inception_block3a_pool = tf.keras.layers.Conv2D(32, (1, 1), padding='SAME', activation='relu')(inception_block3a_pool)

inception_block3a = tf.keras.layers.Concatenate()([inception_block3a_1x1, inception_block3a_3x3,
                                                   inception_block3a_5x5, inception_block3a_pool])

In [6]:
inception_block3b_1x1 = tf.keras.layers.Conv2D(32, (1, 1), strides=(1, 1), padding='SAME', activation='relu')(inception_block3a)

inception_block3b_3x3 = tf.keras.layers.Conv2D(48, (1, 1), padding='SAME', activation='relu')(inception_block3a)
inception_block3b_3x3 = tf.keras.layers.Conv2D(64, (3, 3), padding='SAME', activation='relu')(inception_block3b_3x3)

inception_block3b_5x5 = tf.keras.layers.Conv2D(8, (1, 1), padding='SAME', activation='relu')(inception_block3a)
inception_block3b_5x5 = tf.keras.layers.Conv2D(16, (5, 5), padding='SAME', activation='relu')(inception_block3b_5x5)

inception_block3b_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(inception_block3a)
inception_block3b_pool = tf.keras.layers.Conv2D(16, (1, 1), padding='SAME', activation='relu')(inception_block3b_pool)

inception_block3b = tf.keras.layers.Concatenate()([inception_block3b_1x1, inception_block3b_3x3,
                                                   inception_block3b_5x5, inception_block3b_pool])

In [7]:
max_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='SAME')(inception_block3b)

In [8]:
inception_block4a_1x1 = tf.keras.layers.Conv2D(192, (1, 1), padding='SAME', activation='relu')(max_pool)

inception_block4a_3x3 = tf.keras.layers.Conv2D(96, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block4a_3x3 = tf.keras.layers.Conv2D(208, (3, 3), padding='SAME', activation='relu')(inception_block4a_3x3)

inception_block4a_5x5 = tf.keras.layers.Conv2D(16, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block4a_5x5 = tf.keras.layers.Conv2D(48, (5, 5), padding='SAME', activation='relu')(inception_block4a_5x5)

inception_block4a_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(max_pool)
inception_block4a_pool = tf.keras.layers.Conv2D(64, (1, 1), padding='SAME', activation='relu')(inception_block4a_pool)

inception_block4a = tf.keras.layers.Concatenate()([inception_block4a_1x1, inception_block4a_3x3,
                                                   inception_block4a_5x5, inception_block4a_pool])

In [9]:
inception_block4b_1x1 = tf.keras.layers.Conv2D(160, (1, 1), padding='SAME', activation='relu')(inception_block4a)
inception_block4b_3x3 = tf.keras.layers.Conv2D(112, (1, 1), padding='SAME', activation='relu')(inception_block4a)
inception_block4b_3x3 = tf.keras.layers.Conv2D(224, (3, 3), padding='SAME', activation='relu')(inception_block4b_3x3)

inception_block4b_5x5 = tf.keras.layers.Conv2D(24, (1, 1), padding='SAME', activation='relu')(inception_block4a)
inception_block4b_5x5 = tf.keras.layers.Conv2D(64, (5, 5), padding='SAME', activation='relu')(inception_block4b_5x5)

inception_block4b_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(inception_block4a)
inception_block4b_pool = tf.keras.layers.Conv2D(64, (1, 1), padding='SAME', activation='relu')(inception_block4b_pool)

inception_block4b = tf.keras.layers.Concatenate()([inception_block4b_1x1, inception_block4b_3x3,
                                                   inception_block4b_5x5, inception_block4b_pool])

In [10]:
inception_block4c_1x1 = tf.keras.layers.Conv2D(128, (1, 1), padding='SAME', activation='relu')(inception_block4b)

inception_block4c_3x3 = tf.keras.layers.Conv2D(128, (1, 1), padding='SAME', activation='relu')(inception_block4b)
inception_block4c_3x3 = tf.keras.layers.Conv2D(256, (3, 3), padding='SAME', activation='relu')(inception_block4c_3x3)

inception_block4c_5x5 = tf.keras.layers.Conv2D(24, (1, 1), padding='SAME', activation='relu')(inception_block4b)
inception_block4c_5x5 = tf.keras.layers.Conv2D(64, (5, 5), padding='SAME', activation='relu')(inception_block4c_5x5)

inception_block4c_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(inception_block4b)
inception_block4c_pool = tf.keras.layers.Conv2D(64, (1, 1), padding='SAME', activation='relu')(inception_block4c_pool)

inception_block4c = tf.keras.layers.Concatenate()([inception_block4c_1x1, inception_block4c_3x3,
                                                   inception_block4c_5x5, inception_block4c_pool])

In [11]:
inception_block4d_1x1 = tf.keras.layers.Conv2D(112, (1, 1), padding='SAME', activation='relu')(inception_block4c)

inception_block4d_3x3 = tf.keras.layers.Conv2D(144, (1, 1), padding='SAME', activation='relu')(inception_block4c)
inception_block4d_3x3 = tf.keras.layers.Conv2D(288, (3, 3), padding='SAME', activation='relu')(inception_block4d_3x3)

inception_block4d_5x5 = tf.keras.layers.Conv2D(32, (1, 1), padding='SAME', activation='relu')(inception_block4c)
inception_block4d_5x5 = tf.keras.layers.Conv2D(64, (5, 5), padding='SAME', activation='relu')(inception_block4d_5x5)

inception_block4d_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(inception_block4c)
inception_block4d_pool = tf.keras.layers.Conv2D(64, (1, 1), padding='SAME', activation='relu')(inception_block4d_pool)

inception_block4d = tf.keras.layers.Concatenate()([inception_block4d_1x1, inception_block4d_3x3,
                                                   inception_block4d_5x5, inception_block4d_pool])

In [12]:
inception_block4e_1x1 = tf.keras.layers.Conv2D(256, (1, 1), padding='SAME', activation='relu')(inception_block4d)

inception_block4e_3x3 = tf.keras.layers.Conv2D(160, (1, 1), padding='SAME', activation='relu')(inception_block4d)
inception_block4e_3x3 = tf.keras.layers.Conv2D(320, (3, 3), padding='SAME', activation='relu')(inception_block4e_3x3)

inception_block4e_5x5 = tf.keras.layers.Conv2D(32, (1, 1), padding='SAME', activation='relu')(inception_block4d)
inception_block4e_5x5 = tf.keras.layers.Conv2D(128, (5, 5), padding='SAME', activation='relu')(inception_block4e_5x5)

inception_block4e_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(inception_block4d)
inception_block4e_pool = tf.keras.layers.Conv2D(128, (1, 1), padding='SAME', activation='relu')(inception_block4e_pool)

inception_block4e = tf.keras.layers.Concatenate()([inception_block4e_1x1, inception_block4e_3x3,
                                                   inception_block4e_5x5, inception_block4e_pool])

In [13]:
max_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='SAME')(inception_block4e)

In [14]:
inception_block5a_1x1 = tf.keras.layers.Conv2D(256, (1, 1), padding='SAME', activation='relu')(max_pool)

inception_block5a_3x3 = tf.keras.layers.Conv2D(160, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block5a_3x3 = tf.keras.layers.Conv2D(320, (3, 3), padding='SAME', activation='relu')(inception_block5a_3x3)

inception_block5a_5x5 = tf.keras.layers.Conv2D(32, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block5a_5x5 = tf.keras.layers.Conv2D(128, (5, 5), padding='SAME', activation='relu')(inception_block5a_5x5)

inception_block5a_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(max_pool)
inception_block5a_pool = tf.keras.layers.Conv2D(128, (1, 1), padding='SAME', activation='relu')(inception_block5a_pool)

inception_block5a = tf.keras.layers.Concatenate()([inception_block5a_1x1, inception_block5a_3x3,
                                                   inception_block5a_5x5, inception_block5a_pool])

In [15]:
inception_block5b_1x1 = tf.keras.layers.Conv2D(128, (1, 1), padding='SAME', activation='relu')(inception_block5a)

inception_block5b_3x3 = tf.keras.layers.Conv2D(80, (1, 1), padding='SAME', activation='relu')(inception_block5a)
inception_block5b_3x3 = tf.keras.layers.Conv2D(120, (3, 3), padding='SAME', activation='relu')(inception_block5b_3x3)

inception_block5b_5x5 = tf.keras.layers.Conv2D(16, (1, 1), padding='SAME', activation='relu')(inception_block5a)
inception_block5b_5x5 = tf.keras.layers.Conv2D(64, (5, 5), padding='SAME', activation='relu')(inception_block5b_5x5)

inception_block5b_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(inception_block5a)
inception_block5b_pool = tf.keras.layers.Conv2D(64, (1, 1), padding='SAME', activation='relu')(inception_block5b_pool)

inception_block5b = tf.keras.layers.Concatenate()([inception_block5b_1x1, inception_block5b_3x3,
                                                   inception_block5b_5x5, inception_block5b_pool])

In [16]:
avg_pool = tf.keras.layers.GlobalAveragePooling2D()(inception_block5b) # 416
dropout = tf.keras.layers.Dropout(0.4)(avg_pool)
out_layer = tf.keras.layers.Dense(4, activation='softmax')(dropout)

In [17]:
model = tf.keras.models.Model(input_layer, out_layer)

#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

#model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 227, 227, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 114, 114, 64) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 114, 114, 192 110784      max_pooling2d[0][0]              
______________________________________________________________________________________________

In [18]:
model.fit(train_generator, epochs=50, validation_data=test_generator)

Epoch 1/50
15/15 [==============================] - 59s 4s/step - loss: 1.4253 - accuracy: 0.2000 - val_loss: 1.3864 - val_accuracy: 0.2500
Epoch 2/50
15/15 [==============================] - 52s 3s/step - loss: 1.3870 - accuracy: 0.1833 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 3/50
15/15 [==============================] - 51s 3s/step - loss: 1.3867 - accuracy: 0.2500 - val_loss: 1.3864 - val_accuracy: 0.2500
Epoch 4/50
15/15 [==============================] - 52s 3s/step - loss: 1.3882 - accuracy: 0.2167 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 5/50
15/15 [==============================] - 52s 3s/step - loss: 1.3862 - accuracy: 0.2833 - val_loss: 1.3861 - val_accuracy: 0.2500
Epoch 6/50
15/15 [==============================] - 52s 4s/step - loss: 1.3906 - accuracy: 0.1500 - val_loss: 1.3862 - val_accuracy: 0.2500
Epoch 7/50
15/15 [==============================] - 52s 3s/step - loss: 1.3846 - accuracy: 0.3167 - val_loss: 1.3818 - val_accuracy: 0.2500
Epoch 8/50
15/15 [==

In [19]:
print("-- Evaluate --")
scores = model.evaluate(test_generator)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
3/3 [==============================] - 2s 754ms/step - loss: 0.1322 - accuracy: 1.0000
accuracy: 100.00%


In [20]:
print("-- Predict --")
output = model.predict(test_generator)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

-- Predict --
{'1': 0, '2': 1, '3': 2, '4': 3}
[[0.619 0.013 0.368 0.000]
 [0.088 0.006 0.906 0.000]
 [0.087 0.006 0.907 0.000]
 [0.000 0.000 0.000 1.000]
 [0.086 0.006 0.908 0.000]
 [0.650 0.014 0.336 0.000]
 [0.682 0.014 0.303 0.000]
 [0.000 1.000 0.000 0.000]
 [0.000 0.000 0.000 1.000]
 [0.000 1.000 0.000 0.000]
 [0.000 1.000 0.000 0.000]
 [0.000 0.000 0.000 1.000]]
